In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pprint import pprint

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
train_df

In [ ]:
labelencoder = LabelEncoder()
train_df['thal'] = labelencoder.fit_transform(train_df['thal'])
test_df['thal'] = labelencoder.transform(test_df['thal'])
train_df

In [ ]:
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=133, shuffle= True)
train_labels = train_data.pop('target')
val_labels = val_data.pop('target')
test_labels = test_df.pop('target')

In [ ]:
# train_data.loc[:,['age','trestbps']].describe()
train_data.describe()

In [ ]:
def create_model():
    input1 = tf.keras.Input(shape=(13,))
    hidden1 = tf.keras.layers.Dense(units = 12, activation='relu')(input1)
    hidden2 = tf.keras.layers.Dense(units = 6, activation='relu')(hidden1)
    output1 = tf.keras.layers.Dense(units = 2, activation='softmax')(hidden2)
    
    model = tf.keras.models.Model(inputs = input1, outputs = output1, name= "functional1")
    
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_model1():
    model = tf.keras.Sequential([
    tf.keras.Input(shape = (13,)),
    tf.keras.layers.Dense(units = 12, activation='relu'),
    tf.keras.layers.Dense(units = 6, activation='relu'),
    tf.keras.layers.Dense(units = 2, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_model2():
    input1 = tf.keras.Input(shape=(13,))
    hidden1 = tf.keras.layers.Dense(units = 6, activation='relu')(input1)
    hidden2 = tf.keras.layers.Dense(units = 6, activation='relu')(input1)
    merge = tf.keras.layers.concatenate([hidden1,hidden2])
    hidden3 = tf.keras.layers.Dense(units = 3, activation='relu')(merge)
    output1 = tf.keras.layers.Dense(units = 2, activation='softmax')(hidden3)
    
    model = tf.keras.models.Model(inputs = input1, outputs = output1)
    

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def create_model3():
    input1 = tf.keras.Input(shape=(13,), name = 'I1')
    input2 = tf.keras.Input(shape=(6,), name = 'I2')
    
    hidden1 = tf.keras.layers.Dense(units = 4, activation='relu')(input1)
    hidden2 = tf.keras.layers.Dense(units = 4, activation='relu')(input2)
    merge = tf.keras.layers.concatenate([hidden1, hidden2])
    hidden3 = tf.keras.layers.Dense(units = 3, activation='relu')(merge)
    output1 = tf.keras.layers.Dense(units = 2, activation='softmax', name ='O1')(hidden3)
    output2 = tf.keras.layers.Dense(units = 2, activation='softmax', name = 'O2')(hidden3)
    
    model = tf.keras.models.Model(inputs = [input1,input2], outputs = [output1,output2])
    
    model.compile(optimizer='adam',
                  loss={'O1':'sparse_categorical_crossentropy', 'O2':'mse'},
                  metrics={'O1':'accuracy', 'O2':['mse',tf.keras.metrics.RootMeanSquaredError()]})
    return model



model = create_model3()
tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)
# model.summary()

In [ ]:
new_train_data = train_data.loc[:,['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs']]
new_val_data = val_data.loc[:,['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs']]
new_val_labels = val_data['thalach']
new_train_labels = train_data['thalach']
new_train_data

In [ ]:
layer1 = model.get_layer(index = 2)
sns.heatmap(layer1.weights[0].numpy())

In [ ]:
model.save_weights("initials.h5")

In [ ]:
model.load_weights("initials.h5")
history = model.fit(
    x = train_data, 
    y = train_labels,
    batch_size = 192,
    epochs = 100,
    verbose = 1,
    callbacks = None,
    validation_data = (val_data, val_labels),
    shuffle = True,
    class_weight = {0:0.5, 1:10}
)

In [ ]:
model.load_weights("initials.h5")
history = model.fit(
    x = {'I1':train_data, 'I2':new_train_data}, 
    y = {'O1':train_labels, 'O2': new_train_labels},
    batch_size = 32,
    epochs = 10,
    verbose = 1,
    callbacks = None,
    validation_data = [(val_data,new_val_data),(val_labels, new_val_labels)],
)

In [ ]:
test_loss= model.evaluate(
    x = {'I1':train_data, 'I2':new_train_data}, 
    y = {'O1':train_labels, 'O2': new_train_labels},
    batch_size = 32,
    verbose = 1
)
test_loss

In [ ]:
predictions_probabilities = model.predict(x = {'I1':train_data, 'I2':new_train_data})
predictions_probabilities[0]

In [ ]:
predictions1 = tf.math.argmax(predictions_probabilities[0], 1).numpy()
predictions2 = tf.math.argmax(predictions_probabilities[1], 1).numpy()
predictions2
